In [1]:
import spacy
nlp = spacy.load("fr_core_news_sm")

In [2]:
import numpy as np
import re # regular expressions
from collections import defaultdict
from sklearn.svm import SVC # Support Vector Classification model
from nltk.chat.util import Chat, reflections

In [15]:
pairs=[
         ["qu'est ce que la tuberculose(.*)|ce quoi la tuberculose(.*)|(.*)maladie de la tuberculose",["La tuberculose est une maladie infectieuse causée par les bacilles ou bactéries de la tuberculose (appelés Mycobacterium tuberculosis). La forme de tuberculose la plus répandue est la tuberculose pulmonaire env. 70 pourcents des cas. D’autres formes de tuberculose, telle que la tuberculose lymphatique ou la tuberculose osseuse ne sont pas contagieuses"]],
         ["(.*)transmet|(.*)transmission|(.*)propager|(.*)propagation|(.*)contaminer|(.*)contamination|(.*)contaminé|(.*)transmettre|(.*)propagé",["Les bacilles de la tuberculose se transmettent d’une personne à l’autre par l’air. Lorsqu’elle tousse, la personne tuberculeuse projette dans l’air ambiant de microscopiques gouttelettes qui contiennent des bacilles de la tuberculose. Ces gouttelettes restent un certain temps dans l’air et peuvent donc être inspirées par d’autres personnes. Mais le risque de contamination n’existe qu’en cas de contact avec une personne souffrant d’une tuberculose pulmonaire contagieuse"]],
         ["Quand parle-t-on d’une maladie tuberculeuse|Quand parle-t-on d’une tuberculose|Quand parle-t-on de la tuberculose|quand parle de la tuberculose",["On parle d’une tuberculose lorsque les bacilles de la tuberculose prolifèrent dans les poumons, ce qui peut se produire des mois, voire des années après la contamination. Les bactéries de la tuberculose peuvent aussi toucher d’autres parties du corps par voie sanguine (par exemple les ganglions, le système nerveux central, les os). Ces formes de tuberculose ne sont toutefois pas contagieuses. La tuberculose est surtout dangereuse pour les enfants en bas âge et les personnes dont le système immunitaire est affaibli. Aujourd’hui, la tuberculose a de bonnes chances d’être guérie si elle est traitée correctement."]],
         ["(.*)symptômes|(.*)symptôme|(.*)symptomes|(.*)symptome",["Au début, la maladie provoque peu de symptômes pénibles: toux (parfois sanguinolente), fatigue, légère fièvre et sudation nocturne. A un stade avancé d’autres symptômes peuvent apparaître tels que: perte de poids, perte d’appétit, douleurs dans la poitrine."]],
         ["tuberculose pulmonaire contagieuse|tuberculose pulmonaire",["On parle de tuberculose pulmonaire contagieuse lorsque la maladie a progressé au point que la personne atteinte expectore des bacilles tuberculeux lorsqu’elle tousse (ou qu’elle éternue). Ce n’est qu’à ce stade de la maladie que la personne tuberculeuse peut contaminer d’autres personnes"]],
         ["temps de contamination|temps pour contaminer|période contamination|durée pour contaminer",["Une personne souffrant d’une tuberculose contagieuse est placée en isolement à l’hôpital ou à son domicile, ce qui signifie qu’elle est écartée de son entourage. Elle reste en isolement jusqu’à ce qu’elle ne soit plus contagieuse, à savoir en règle générale deux à trois semaines. Passé ce délai, elle n’est plus contagieuse. Mais elle doit poursuivre son traitement, sous contrôle médical, pendant environ six mois ou davantage jusqu’à ce qu’elle soit totalement guérie."]],
         ["(.*)être contaminé(.*)|(.*)de contamination(.*)|(.*)etre  contaminer(.*)|(.*)etre contaminé(.*)|(.*)être contaminer(.*)",["Seules les personnes qui ont un contact très étroit (plusieurs heures dans la même pièce) avec une personne tuberculeuse et contagieuse courent un risque de contamination"]],
         ["(.*)vaccin|(.*)vaccination",["les vaccinations contre la tuberculose (BCG) d’autrefois n’offrent qu’une protection très limitée"]],
         ["(.*)contacte avec un malade(.*)|(.*)précaution après contacte avec un malade(.*)|(.*)après contacte avec un malade(.*)|(.*)après contact(.*)|(.*)contacte avec un malade(.*)|(.*)après etre en contact(.*)|(.*)être en contact(.*)|(.*)etre en contact(.*)|(.*)été en contact(.*)|(.*)être en contacte(.*)|(.*)été en contacte(.*)",["Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous prodiguera gratuitement des conseils et des informations concernant tous les aspects de la tuberculose. Si vous avez été en contact très étroit avec une personne souffrant d’une tuberculose pulmonaire contagieuse (question 6), vous pouvez prendre rendez-vous avec ce service (adresses en annexe) pour vous informer et effectuer un test qui montrera si vous avez été contaminé"]],
         ["(.*)contagion",["Une contagion peut être mise en évidence au moyen d’un test cutané à la tuberculine ou d’un test sanguin(appelé aussi test interféron gamma).Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous fournira volontiers des informations sur ces tests et sur l’interprétation de leurs résultats."]],
         ["(.*)test cutané(.*)|(.*)test sanguin(.*)",["Les bacilles de la tuberculose ne se propagent que très lentement et une réaction immunologique n’apparaît qu’après plusieurs semaines. C’est pourquoi ce n’est qu’après huit semaines que l’on peut confirmer une contamination. Chez les enfants âgés de moins de douze ans, il existe un risque plus élevé qu’ils développent des formes dangereuses de tuberculose. Par ailleurs, à cet âge, la maladie peut progresser plus rapidement.C’est pourquoi, par simple mesure de sécurité, on procède chez eux à un test immédiatement après le dernier contact; selon le résultat du test, on le répète après huit autres semaines."]],
         ["(.*)résultat du test sanguin est positif(.*)|(.*)résultat positif(.*)|(.*)testé positif(.*)|(.*)tester positif(.*)|(.*)résultat du test sanguin(.*)",["Un résultat de test positif signifie qu’un contact avec des bacilles tuberculeux a eu lieu. Le service spécialisé pour la tuberculose vous en informera ainsi que votre médecin traitant en vous demandant de convenir d’une date pour effectuer d’autres examens (radiographie, etc.) pour pouvoir confirmer ou exclure une tuberculose."]],
         ["(.*)J’ai été contaminé(.*)",["Non! Une contamination ne signifie ni qu’on souffre de tuberculose, ni qu’on est contagieux. Ainsi, même si votre test sanguin est positif, vos proches ne sont pas en danger."]],
         ["(.*)traité(.*)|(.*)traitement(.*)",["Une contamination prouvée est traitée par un antibiotique. Selon l’antibiotique utilisé, le traitement s’effectue sur une durée de quatre à neuf mois. Le traitement est recommandé, parce qu’il réduit de manière significative le risque de développer ultérieurement une tuberculose. La personne contaminée décide avec son médecin traitant si un tel traitement doit être effectué."]],
         ["(.*)prise en charge(.*)",["Dans le domaine de la tuberculose, la prise en charge des frais est régie de manière différente selon les cantons. Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous informera volontiers des dispositions applicables dans votre canton de domicile."]],
         ["bonjour|salut|(.*)tu|(.*)vous|(.*)comment|bonsoir",["Salut"]]
         
         ]

In [3]:
output_format = "IN: {input}\nOUT: {output}\n" + "_"*50

In [4]:
# hard-coded exact questions
responses_exact = {
    "qu'est ce que la tuberculose?": "La tuberculose est une maladie infectieuse causée par les bacilles ou bactéries de la tuberculose (appelés Mycobacterium tuberculosis). La forme de tuberculose la plus répandue est la tuberculose pulmonaire env. 70 pourcents des cas. D’autres formes de tuberculose, telle que la tuberculose lymphatique ou la tuberculose osseuse ne sont pas contagieuses!",
    "comment se transmet-elle?": "Les bacilles de la tuberculose se transmettent d’une personne à l’autre par l’air. Lorsqu’elle tousse, la personne tuberculeuse projette dans l’air ambiant de microscopiques gouttelettes qui contiennent des bacilles de la tuberculose. Ces gouttelettes restent un certain temps dans l’air et peuvent donc être inspirées par d’autres personnes. Mais le risque de contamination n’existe qu’en cas de contact avec une personne souffrant d’une tuberculose pulmonaire contagieuse.",
    "Quand parle-t-on d’une maladie tuberculeuse?": "On parle d’une tuberculose lorsque les bacilles de la tuberculose prolifèrent dans les poumons, ce qui peut se produire des mois, voire des années après la contamination. Les bactéries de la tuberculose peuvent aussi toucher d’autres parties du corps par voie sanguine (par exemple les ganglions, le système nerveux central, les os). Ces formes de tuberculose ne sont toutefois pas contagieuses. La tuberculose est surtout dangereuse pour les enfants en bas âge et les personnes dont le système immunitaire est affaibli. Aujourd’hui, la tuberculose a de bonnes chances d’être guérie si elle est traitée correctement.",
    "quel sont les symptômes?": "Au début, la maladie provoque peu de symptômes pénibles: toux (parfois sanguinolente), fatigue, légère fièvre et sudation nocturne. A un stade avancé d’autres symptômes peuvent apparaître tels que: perte de poids, perte d’appétit, douleurs dans la poitrine.",
    "ce quoi la tuberculose pulmonaire contagieuse?": "On parle de tuberculose pulmonaire contagieuse lorsque la maladie a progressé au point que la personne atteinte expectore des bacilles tuberculeux lorsqu’elle tousse (ou qu’elle éternue). Ce n’est qu’à ce stade de la maladie que la personne tuberculeuse peut contaminer d’autres personnes.",
    "quel est le temps de la contamination?": "Une personne souffrant d’une tuberculose contagieuse est placée en isolement à l’hôpital ou à son domicile, ce qui signifie qu’elle est écartée de son entourage. Elle reste en isolement jusqu’à ce qu’elle ne soit plus contagieuse, à savoir en règle générale deux à trois semaines. Passé ce délai, elle n’est plus contagieuse. Mais elle doit poursuivre son traitement, sous contrôle médical, pendant environ six mois ou davantage jusqu’à ce qu’elle soit totalement guérie.",
    "qui peut être contaminé?": "Seules les personnes qui ont un contact très étroit (plusieurs heures dans la même pièce) avec une personne tuberculeuse et contagieuse courent un risque de contamination.",
    "Existe - il un vaccin?": "les vaccinations contre la tuberculose (BCG) d’autrefois n’offrent qu’une protection très limitée.",
    "Quel sont les précautions après contacte avec un malade?": "Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous prodiguera gratuitement des conseils et des informations concernant tous les aspects de la tuberculose. Si vous avez été en contact très étroit avec une personne souffrant d’une tuberculose pulmonaire contagieuse, vous pouvez prendre rendez-vous avec ce service  pour vous informer et effectuer un test qui montrera si vous avez été contaminé.",
    "contagion?": "Une contagion peut être mise en évidence au moyen d’un test cutané à la tuberculine ou d’un test sanguin(appelé aussi test interféron gamma).Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous fournira volontiers des informations sur ces tests et sur l’interprétation de leurs résultats.",
    "ce quoi test cutané?": "Les bacilles de la tuberculose ne se propagent que très lentement et une réaction immunologique n’apparaît qu’après plusieurs semaines. C’est pourquoi ce n’est qu’après huit semaines que l’on peut confirmer une contamination. Chez les enfants âgés de moins de douze ans, il existe un risque plus élevé qu’ils développent des formes dangereuses de tuberculose. Par ailleurs, à cet âge, la maladie peut progresser plus rapidement.C’est pourquoi, par simple mesure de sécurité, on procède chez eux à un test immédiatement après le dernier contact; selon le résultat du test, on le répète après huit autres semaines.",
    "résultat du test sanguin est positif?": "Un résultat de test positif signifie qu’un contact avec des bacilles tuberculeux a eu lieu. Le service spécialisé pour la tuberculose vous en informera ainsi que votre médecin traitant en vous demandant de convenir d’une date pour effectuer d’autres examens (radiographie, etc.) pour pouvoir confirmer ou exclure une tuberculose.",
    "J’ai été contaminé?": "Non! Une contamination ne signifie ni qu’on souffre de tuberculose, ni qu’on est contagieux. Ainsi, même si votre test sanguin est positif, vos proches ne sont pas en danger.",
    "existe il un traitement?": "Une contamination prouvée est traitée par un antibiotique. Selon l’antibiotique utilisé, le traitement s’effectue sur une durée de quatre à neuf mois. Le traitement est recommandé, parce qu’il réduit de manière significative le risque de développer ultérieurement une tuberculose. La personne contaminée décide avec son médecin traitant si un tel traitement doit être effectué.",
    "Comment se fait la prise en charge?": "Dans le domaine de la tuberculose, la prise en charge des frais est régie de manière différente selon les cantons. Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous informera volontiers des dispositions applicables dans votre canton de domicile.",
    "Bonjour?": "salut.",
    "default": "je vous ai pas compris!"
}

def respond_exact(text):
    response = responses_exact.get(text.lower(), responses_exact['default'])
    return(output_format.format(input=text, output=response))

In [5]:
print(respond_exact("qu'est ce que la tuberculose?"))
print("_"*50)
print(respond_exact("comment se transmet-elle?"))
print("_"*50)
print(respond_exact("Quand parle-t-on d’une maladie tuberculeuse?"))
print("_"*50)
print(respond_exact("quel sont les symptômes?"))
print("_"*50)
print(respond_exact("ce quoi la tuberculose pulmonaire contagieuse?"))
print("_"*50)
print(respond_exact("quel est le temps de la contamination?"))
print("_"*50)
print(respond_exact("qui peut être contaminé?"))
print("_"*50)
print(respond_exact("Existe - il un vaccin?"))
print("_"*50)
print(respond_exact("Quel sont les précautions après contacte avec un malade?"))
print("_"*50)
print(respond_exact("contagion?"))
print("_"*50)
print(respond_exact("ce quoi test cutané?"))
print("_"*50)
print(respond_exact("résultat du test sanguin est positif?"))
print("_"*50)
print(respond_exact("J’ai été contaminé?"))
print("_"*50)
print(respond_exact("existe il un traitement?"))
print("_"*50)
print(respond_exact("Comment se fait la prise en charge?"))
print("_"*50)
print(respond_exact("Bonjour?"))
print("_"*50)
print(respond_exact("je suis en couple."))

IN: qu'est ce que la tuberculose?
OUT: La tuberculose est une maladie infectieuse causée par les bacilles ou bactéries de la tuberculose (appelés Mycobacterium tuberculosis). La forme de tuberculose la plus répandue est la tuberculose pulmonaire env. 70 pourcents des cas. D’autres formes de tuberculose, telle que la tuberculose lymphatique ou la tuberculose osseuse ne sont pas contagieuses!
__________________________________________________
__________________________________________________
IN: comment se transmet-elle?
OUT: Les bacilles de la tuberculose se transmettent d’une personne à l’autre par l’air. Lorsqu’elle tousse, la personne tuberculeuse projette dans l’air ambiant de microscopiques gouttelettes qui contiennent des bacilles de la tuberculose. Ces gouttelettes restent un certain temps dans l’air et peuvent donc être inspirées par d’autres personnes. Mais le risque de contamination n’existe qu’en cas de contact avec une personne souffrant d’une tuberculose pulmonaire contagi

In [6]:
# Définissez des mots clés qui peuvent aider à déterminer l'intention 
intent_keywords = {
    'motif_1': ['ce quoi la tuberculose', 'maladie de la tuberculose','la tuberculose'],
    'motif_2': ['transmet', 'transmission', 'propager', 'propagation','contaminer','contamination','contaminé','transmettre','propagé'],
    'motif_3': ['Quand'],
    'motif_4': ['symptômes', 'symptôme', 'symptomes', 'symptome'],
    
    'motif_5': ['tuberculose pulmonaire contagieuse', 'tuberculose pulmonaire'],
    'motif_6': ['temps de contamination', 'temps pour contaminer', 'période contamination', 'durée pour contaminer'],
    
    'motif_7': ['être contaminé', 'de contamination'],
    'motif_8': ['vaccin', 'vaccination'],
    
    'motif_9': ['précaution', 'contact', 'en contact'],
    'motif_10': ['contagion'],
    
    'motif_11': ['test'],
    'motif_12': ['résultat', 'positif'],
    
    'motif_13': ['proches', 'proche'],
    'motif_14': ['traité', 'traitement'],
    
    'motif_15': ['prise en charge', 'en charge'],
    'motif_16': ['bonjour', 'salut', 'bonsoir', 'ca vas'],
}
# Create a dictionary of patterns
patterns = {intent: re.compile('|'.join(keys)) for intent, keys in intent_keywords.items()}

# Define a function to find the intent of a message
def get_intent_re(message):
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message):
            return(intent)
    else:
        return('default')

responses_re = {

    
    "motif_1": "La tuberculose est une maladie infectieuse causée par les bacilles ou bactéries de la tuberculose (appelés Mycobacterium tuberculosis). La forme de tuberculose la plus répandue est la tuberculose pulmonaire env. 70 pourcents des cas. D’autres formes de tuberculose, telle que la tuberculose lymphatique ou la tuberculose osseuse ne sont pas contagieuses!",
    "motif_2": "Les bacilles de la tuberculose se transmettent d’une personne à l’autre par l’air. Lorsqu’elle tousse, la personne tuberculeuse projette dans l’air ambiant de microscopiques gouttelettes qui contiennent des bacilles de la tuberculose. Ces gouttelettes restent un certain temps dans l’air et peuvent donc être inspirées par d’autres personnes. Mais le risque de contamination n’existe qu’en cas de contact avec une personne souffrant d’une tuberculose pulmonaire contagieuse.",
    "motif_3": "On parle d’une tuberculose lorsque les bacilles de la tuberculose prolifèrent dans les poumons, ce qui peut se produire des mois, voire des années après la contamination. Les bactéries de la tuberculose peuvent aussi toucher d’autres parties du corps par voie sanguine (par exemple les ganglions, le système nerveux central, les os). Ces formes de tuberculose ne sont toutefois pas contagieuses. La tuberculose est surtout dangereuse pour les enfants en bas âge et les personnes dont le système immunitaire est affaibli. Aujourd’hui, la tuberculose a de bonnes chances d’être guérie si elle est traitée correctement.",
    "motif_4": "Au début, la maladie provoque peu de symptômes pénibles: toux (parfois sanguinolente), fatigue, légère fièvre et sudation nocturne. A un stade avancé d’autres symptômes peuvent apparaître tels que: perte de poids, perte d’appétit, douleurs dans la poitrine.",
    "motif_5": "On parle de tuberculose pulmonaire contagieuse lorsque la maladie a progressé au point que la personne atteinte expectore des bacilles tuberculeux lorsqu’elle tousse (ou qu’elle éternue). Ce n’est qu’à ce stade de la maladie que la personne tuberculeuse peut contaminer d’autres personnes.",
    "motif_6": "Une personne souffrant d’une tuberculose contagieuse est placée en isolement à l’hôpital ou à son domicile, ce qui signifie qu’elle est écartée de son entourage. Elle reste en isolement jusqu’à ce qu’elle ne soit plus contagieuse, à savoir en règle générale deux à trois semaines. Passé ce délai, elle n’est plus contagieuse. Mais elle doit poursuivre son traitement, sous contrôle médical, pendant environ six mois ou davantage jusqu’à ce qu’elle soit totalement guérie.",
    "motif_7": "Seules les personnes qui ont un contact très étroit (plusieurs heures dans la même pièce) avec une personne tuberculeuse et contagieuse courent un risque de contamination.",
    "motif_8": "les vaccinations contre la tuberculose (BCG) d’autrefois n’offrent qu’une protection très limitée.",
    "motif_9": "Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous prodiguera gratuitement des conseils et des informations concernant tous les aspects de la tuberculose. Si vous avez été en contact très étroit avec une personne souffrant d’une tuberculose pulmonaire contagieuse, vous pouvez prendre rendez-vous avec ce service  pour vous informer et effectuer un test qui montrera si vous avez été contaminé.",
    "motif_10": "Une contagion peut être mise en évidence au moyen d’un test cutané à la tuberculine ou d’un test sanguin(appelé aussi test interféron gamma).Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous fournira volontiers des informations sur ces tests et sur l’interprétation de leurs résultats.",
    "motif_11": "Les bacilles de la tuberculose ne se propagent que très lentement et une réaction immunologique n’apparaît qu’après plusieurs semaines. C’est pourquoi ce n’est qu’après huit semaines que l’on peut confirmer une contamination. Chez les enfants âgés de moins de douze ans, il existe un risque plus élevé qu’ils développent des formes dangereuses de tuberculose. Par ailleurs, à cet âge, la maladie peut progresser plus rapidement.C’est pourquoi, par simple mesure de sécurité, on procède chez eux à un test immédiatement après le dernier contact; selon le résultat du test, on le répète après huit autres semaines.",
    "motif_12": "Un résultat de test positif signifie qu’un contact avec des bacilles tuberculeux a eu lieu. Le service spécialisé pour la tuberculose vous en informera ainsi que votre médecin traitant en vous demandant de convenir d’une date pour effectuer d’autres examens (radiographie, etc.) pour pouvoir confirmer ou exclure une tuberculose.",
    "motif_13": "Non! Une contamination ne signifie ni qu’on souffre de tuberculose, ni qu’on est contagieux. Ainsi, même si votre test sanguin est positif, vos proches ne sont pas en danger.",
    "motif_14": "Une contamination prouvée est traitée par un antibiotique. Selon l’antibiotique utilisé, le traitement s’effectue sur une durée de quatre à neuf mois. Le traitement est recommandé, parce qu’il réduit de manière significative le risque de développer ultérieurement une tuberculose. La personne contaminée décide avec son médecin traitant si un tel traitement doit être effectué.",
    "motif_15": "Dans le domaine de la tuberculose, la prise en charge des frais est régie de manière différente selon les cantons. Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous informera volontiers des dispositions applicables dans votre canton de domicile.",
    "motif_16": "salut.",
    "default": "je vous ai pas compris!"
    
    
    
    
    
}

def respond_re(text):
    response = responses_re.get(get_intent_re(text))
    return(output_format.format(input=text, output=response))

In [7]:
print(respond_re("ce quoi la contagion?"))
print(respond_re("c'est quoi la tuberculose?"))
print(respond_re("bonsoir"))

IN: ce quoi la contagion?
OUT: Une contagion peut être mise en évidence au moyen d’un test cutané à la tuberculine ou d’un test sanguin(appelé aussi test interféron gamma).Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous fournira volontiers des informations sur ces tests et sur l’interprétation de leurs résultats.
__________________________________________________
IN: c'est quoi la tuberculose?
OUT: La tuberculose est une maladie infectieuse causée par les bacilles ou bactéries de la tuberculose (appelés Mycobacterium tuberculosis). La forme de tuberculose la plus répandue est la tuberculose pulmonaire env. 70 pourcents des cas. D’autres formes de tuberculose, telle que la tuberculose lymphatique ou la tuberculose osseuse ne sont pas contagieuses!
__________________________________________________
IN: bonsoir
OUT: salut.
__________________________________________________


In [8]:
# Create training data
training_sentences = [

    "qu'est ce que la tuberculose?",
    "comment se transmet-elle?",
    "Quand parle-t-on d’une maladie tuberculeuse?",
    "quel sont les symptômes?",
    "ce quoi la tuberculose pulmonaire contagieuse?",
    "quel est le temps de la contamination?",
    "qui peut être contaminé?",
    "Existe - il un vaccin?",
    "Quel sont les précautions après contacte avec un malade?",
    "contagion?",
    "ce quoi test cutané?",
    "résultat du test sanguin est positif?",
    "J’ai été contaminé?",
    "existe il un traitement?",
    "Comment se fait la prise en charge?",
    "Bonjour?"   
    
]
training_intents = [
    "motif_1",
     "motif_2",
     "motif_3",
     "motif_4",
     "motif_5",
     "motif_6",
     "motif_7",
     "motif_8",
     "motif_9",
     "motif_10",
     "motif_11",
     "motif_12",
     "motif_13",
     "motif_14",
     "motif_15",
     "motif_16"
    
   
]

In [9]:
import spacy.cli
spacy.cli.download("fr_core_news_lg")
import fr_core_news_lg
nlp = fr_core_news_lg.load()


✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_lg')


In [10]:
# Initialize the array with zeros: X
X_train = np.zeros((len(training_sentences), 
              nlp('sentences').vocab.vectors_length))

for i, sentence in enumerate(training_sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X_train[i, :] = doc.vector

In [11]:
# Create a support vector classifier
clf = SVC(C=1, gamma="auto", probability=True)

# Fit the classifier using the training data
clf.fit(X_train, training_intents)

#Yes, a lot can be done here to check / improve model performance! We will leave that for another day!

SVC(C=1, gamma='auto', probability=True)

In [12]:
def get_intent_ml(text):
    doc = nlp(text)
    return(clf.predict([doc.vector])[0])

In [13]:
responses_ml = {

    "motif_1": "La tuberculose est une maladie infectieuse causée par les bacilles ou bactéries de la tuberculose (appelés Mycobacterium tuberculosis). La forme de tuberculose la plus répandue est la tuberculose pulmonaire env. 70 pourcents des cas. D’autres formes de tuberculose, telle que la tuberculose lymphatique ou la tuberculose osseuse ne sont pas contagieuses!",
    "motif_2": "Les bacilles de la tuberculose se transmettent d’une personne à l’autre par l’air. Lorsqu’elle tousse, la personne tuberculeuse projette dans l’air ambiant de microscopiques gouttelettes qui contiennent des bacilles de la tuberculose. Ces gouttelettes restent un certain temps dans l’air et peuvent donc être inspirées par d’autres personnes. Mais le risque de contamination n’existe qu’en cas de contact avec une personne souffrant d’une tuberculose pulmonaire contagieuse.",
    "motif_3": "On parle d’une tuberculose lorsque les bacilles de la tuberculose prolifèrent dans les poumons, ce qui peut se produire des mois, voire des années après la contamination. Les bactéries de la tuberculose peuvent aussi toucher d’autres parties du corps par voie sanguine (par exemple les ganglions, le système nerveux central, les os). Ces formes de tuberculose ne sont toutefois pas contagieuses. La tuberculose est surtout dangereuse pour les enfants en bas âge et les personnes dont le système immunitaire est affaibli. Aujourd’hui, la tuberculose a de bonnes chances d’être guérie si elle est traitée correctement.",
    "motif_4": "Au début, la maladie provoque peu de symptômes pénibles: toux (parfois sanguinolente), fatigue, légère fièvre et sudation nocturne. A un stade avancé d’autres symptômes peuvent apparaître tels que: perte de poids, perte d’appétit, douleurs dans la poitrine.",
    "motif_5": "On parle de tuberculose pulmonaire contagieuse lorsque la maladie a progressé au point que la personne atteinte expectore des bacilles tuberculeux lorsqu’elle tousse (ou qu’elle éternue). Ce n’est qu’à ce stade de la maladie que la personne tuberculeuse peut contaminer d’autres personnes.",
    "motif_6": "Une personne souffrant d’une tuberculose contagieuse est placée en isolement à l’hôpital ou à son domicile, ce qui signifie qu’elle est écartée de son entourage. Elle reste en isolement jusqu’à ce qu’elle ne soit plus contagieuse, à savoir en règle générale deux à trois semaines. Passé ce délai, elle n’est plus contagieuse. Mais elle doit poursuivre son traitement, sous contrôle médical, pendant environ six mois ou davantage jusqu’à ce qu’elle soit totalement guérie.",
    "motif_7": "Seules les personnes qui ont un contact très étroit (plusieurs heures dans la même pièce) avec une personne tuberculeuse et contagieuse courent un risque de contamination.",
    "motif_8": "les vaccinations contre la tuberculose (BCG) d’autrefois n’offrent qu’une protection très limitée.",
    "motif_9": "Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous prodiguera gratuitement des conseils et des informations concernant tous les aspects de la tuberculose. Si vous avez été en contact très étroit avec une personne souffrant d’une tuberculose pulmonaire contagieuse, vous pouvez prendre rendez-vous avec ce service  pour vous informer et effectuer un test qui montrera si vous avez été contaminé.",
    "motif_10": "Une contagion peut être mise en évidence au moyen d’un test cutané à la tuberculine ou d’un test sanguin(appelé aussi test interféron gamma).Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous fournira volontiers des informations sur ces tests et sur l’interprétation de leurs résultats.",
    "motif_11": "Les bacilles de la tuberculose ne se propagent que très lentement et une réaction immunologique n’apparaît qu’après plusieurs semaines. C’est pourquoi ce n’est qu’après huit semaines que l’on peut confirmer une contamination. Chez les enfants âgés de moins de douze ans, il existe un risque plus élevé qu’ils développent des formes dangereuses de tuberculose. Par ailleurs, à cet âge, la maladie peut progresser plus rapidement.C’est pourquoi, par simple mesure de sécurité, on procède chez eux à un test immédiatement après le dernier contact; selon le résultat du test, on le répète après huit autres semaines.",
    "motif_12": "Un résultat de test positif signifie qu’un contact avec des bacilles tuberculeux a eu lieu. Le service spécialisé pour la tuberculose vous en informera ainsi que votre médecin traitant en vous demandant de convenir d’une date pour effectuer d’autres examens (radiographie, etc.) pour pouvoir confirmer ou exclure une tuberculose.",
    "motif_13": "Non! Une contamination ne signifie ni qu’on souffre de tuberculose, ni qu’on est contagieux. Ainsi, même si votre test sanguin est positif, vos proches ne sont pas en danger.",
    "motif_14": "Une contamination prouvée est traitée par un antibiotique. Selon l’antibiotique utilisé, le traitement s’effectue sur une durée de quatre à neuf mois. Le traitement est recommandé, parce qu’il réduit de manière significative le risque de développer ultérieurement une tuberculose. La personne contaminée décide avec son médecin traitant si un tel traitement doit être effectué.",
    "motif_15": "Dans le domaine de la tuberculose, la prise en charge des frais est régie de manière différente selon les cantons. Le service spécialisé pour la tuberculose de la Ligue pulmonaire cantonale vous informera volontiers des dispositions applicables dans votre canton de domicile.",
    "motif_16": "salut",
    "default": "je vous ai pas compris!"   
    
}

def respond_ml(text):
    response = responses_ml.get(get_intent_ml(text), responses_ml["default"])
    return(output_format.format(input=text, output=response))

In [14]:
print(respond_ml("la contamination?"))
print(respond_ml("la tuberculose ce quoi?"))

IN: la contamination?
OUT: Une personne souffrant d’une tuberculose contagieuse est placée en isolement à l’hôpital ou à son domicile, ce qui signifie qu’elle est écartée de son entourage. Elle reste en isolement jusqu’à ce qu’elle ne soit plus contagieuse, à savoir en règle générale deux à trois semaines. Passé ce délai, elle n’est plus contagieuse. Mais elle doit poursuivre son traitement, sous contrôle médical, pendant environ six mois ou davantage jusqu’à ce qu’elle soit totalement guérie.
__________________________________________________
IN: la tuberculose ce quoi?
OUT: On parle de tuberculose pulmonaire contagieuse lorsque la maladie a progressé au point que la personne atteinte expectore des bacilles tuberculeux lorsqu’elle tousse (ou qu’elle éternue). Ce n’est qu’à ce stade de la maladie que la personne tuberculeuse peut contaminer d’autres personnes.
__________________________________________________


In [ ]:
chat=Chat(pairs)
chat.converse()


>qu'est ce que la tuberculose
La tuberculose est une maladie infectieuse causée par les bacilles ou bactéries de la tuberculose (appelés Mycobacterium tuberculosis). La forme de tuberculose la plus répandue est la tuberculose pulmonaire env. 70 pourcents des cas. D’autres formes de tuberculose, telle que la tuberculose lymphatique ou la tuberculose osseuse ne sont pas contagieuses
>mode de transmission
Les bacilles de la tuberculose se transmettent d’une personne à l’autre par l’air. Lorsqu’elle tousse, la personne tuberculeuse projette dans l’air ambiant de microscopiques gouttelettes qui contiennent des bacilles de la tuberculose. Ces gouttelettes restent un certain temps dans l’air et peuvent donc être inspirées par d’autres personnes. Mais le risque de contamination n’existe qu’en cas de contact avec une personne souffrant d’une tuberculose pulmonaire contagieuse
>temps d'incubation
None
>inccubation
None
